In [7]:
import os
os.chdir('/Users/caiorocha12/Desktop/UCF/Fall25/Data Science Programming/FInal Project')

**Library Imports**

In [8]:
import pandas as pd
import altair as alt
import datetime as dt

**.csv Imports**

In [9]:
feat = pd.read_csv('./final_features.csv')
print(feat.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68 entries, 0 to 67
Data columns (total 41 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             68 non-null     object 
 1   AMZN_ret               68 non-null     float64
 2   META_ret               68 non-null     float64
 3   GOOGL_ret              68 non-null     float64
 4   NVDA_ret               68 non-null     float64
 5   MSFT_ret               68 non-null     float64
 6   AMZN_excess            68 non-null     float64
 7   META_excess            68 non-null     float64
 8   GOOGL_excess           68 non-null     float64
 9   NVDA_excess            68 non-null     float64
 10  MSFT_excess            68 non-null     float64
 11  AI                     68 non-null     float64
 12  Gemini                 68 non-null     float64
 13  ChatGPT                68 non-null     float64
 14  Machine Learning       68 non-null     float64
 15  OpenAI  

In [10]:
mkt = pd.read_csv('./market_monthly.csv')
print(mkt.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70 entries, 0 to 69
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Date             70 non-null     object 
 1   ^GSPC_Open       70 non-null     float64
 2   ^GSPC_High       70 non-null     float64
 3   ^GSPC_Low        70 non-null     float64
 4   ^GSPC_Close      70 non-null     float64
 5   ^GSPC_Adj Close  70 non-null     float64
 6   ^GSPC_Volume     70 non-null     int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 4.0+ KB
None


In [11]:
stocks = pd.read_csv('./stocks_monthly.csv')
print(stocks.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70 entries, 0 to 69
Data columns (total 31 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Date             70 non-null     object 
 1   GOOGL_Open       70 non-null     float64
 2   GOOGL_High       70 non-null     float64
 3   GOOGL_Low        70 non-null     float64
 4   GOOGL_Close      70 non-null     float64
 5   GOOGL_Adj Close  70 non-null     float64
 6   GOOGL_Volume     70 non-null     int64  
 7   AMZN_Open        70 non-null     float64
 8   AMZN_High        70 non-null     float64
 9   AMZN_Low         70 non-null     float64
 10  AMZN_Close       70 non-null     float64
 11  AMZN_Adj Close   70 non-null     float64
 12  AMZN_Volume      70 non-null     int64  
 13  MSFT_Open        70 non-null     float64
 14  MSFT_High        70 non-null     float64
 15  MSFT_Low         70 non-null     float64
 16  MSFT_Close       70 non-null     float64
 17  MSFT_Adj Close   7

In [12]:
trends = pd.read_csv('./trends_monthly.csv')
print(trends.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70 entries, 0 to 69
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   date              70 non-null     object
 1   AI                70 non-null     int64 
 2   Gemini            70 non-null     int64 
 3   ChatGPT           70 non-null     int64 
 4   Machine Learning  70 non-null     int64 
 5   OpenAI            70 non-null     int64 
 6   Meta AI           70 non-null     int64 
 7   Microsoft AI      70 non-null     int64 
 8   NVIDIA AI         70 non-null     int64 
 9   Google AI         70 non-null     int64 
 10  Amazon AI         70 non-null     int64 
dtypes: int64(10), object(1)
memory usage: 6.1+ KB
None


**Cleaning DataFrames**

In [13]:
# renaming unnamed column in feat
feat.rename(columns = {'Unnamed: 0': 'Date'}, inplace = True)

In [14]:
# dropping rows from mkt, stocks, and trends that are not in feat
for df in [mkt, stocks, trends]:
    df = df.loc[2:]

mkt.reset_index(inplace = True, drop = True)
stocks.reset_index(inplace = True, drop = True)
trends.reset_index(inplace = True, drop = True)

In [15]:
# creating new col in trends for the average of the row
trends['average'] = trends.drop(columns = 'date').mean(axis = 1)
print(trends.head())

         date  AI  Gemini  ChatGPT  Machine Learning  OpenAI  Meta AI  \
0  2020-01-01   5       2        0                 1       0        0   
1  2020-02-01   5       2        0                 1       0        0   
2  2020-03-01   5       1        0                 0       0        0   
3  2020-04-01   6       2        0                 0       0        0   
4  2020-05-01   5       2        0                 1       0        0   

   Microsoft AI  NVIDIA AI  Google AI  Amazon AI  average  
0             0          0          1          0      0.9  
1             0          0          1          0      0.9  
2             0          0          1          0      0.7  
3             0          0          1          0      0.9  
4             0          0          1          0      0.9  


In [16]:
# creating new df of each stock's adjusted close column scaled to the range [0,1].
# necessary for visualizing relationship between google search trends vs stock price
def scale_col(s):
    min_val = s.min()
    max_val = s.max()
    return s.map(lambda x: (x - min_val)/(max_val - min_val))

scaled = pd.DataFrame()

scaled['^GSPC'] = scale_col(mkt['^GSPC_Adj Close'])
for col in stocks.columns:
    if 'Adj' in col:
        scaled[col[:4]] = scale_col(stocks[col])

scaled = scaled * 100
scaled['Date'] = pd.to_datetime(stocks['Date'])
print(scaled)

         ^GSPC        GOOG        AMZN        MSFT        META        NVDA  \
0    15.060824    6.017593   10.258393    3.008965   15.886973    0.000000   
1     8.685708    3.939441    6.358445    0.963041   14.507911    0.425883   
2     0.000000    0.000000    8.417176    0.000000   10.757857    0.345968   
3     7.703710    4.104974   24.778428    5.365889   16.296020    0.709274   
4    10.802681    6.034033   23.791349    6.374157   19.273270    1.504037   
..         ...         ...         ...         ...         ...         ...   
65   85.072646   52.870308   84.502558   90.572602   94.724143   77.355696   
66   88.231768   59.970781   93.689926  100.000000  100.000000   87.472339   
67   91.072010   69.363229   90.500561   92.996718   94.897094   85.595581   
68   96.434348   82.859566   84.614908   96.160466   94.262537   91.902292   
69  100.000000  100.000000  100.000000   96.123832   81.689986  100.000000   

         Date  
0  2020-01-01  
1  2020-02-01  
2  2020-03-01  

**Visualizations**

In [17]:
trend_long = pd.DataFrame({'Date': pd.to_datetime(trends['date']),
        'Symbol': 'Average Google Trend',
        'Value': trends['average']})

scaled_long = scaled.copy().melt(id_vars = 'Date',
        var_name = 'Symbol',
        value_name = 'Value')

long = pd.concat([trend_long, scaled_long], ignore_index=True)

scaled_ser = scaled_long['Symbol'].unique().tolist()

select_ser = alt.selection_point(fields = ['Symbol'], on = 'pointerover', toggle = True)

In [18]:
trend_vis = alt.Chart(long[long['Symbol'] == 'Average Google Trend']).mark_line(strokeWidth = 4).encode(
        x = alt.X('Date:T', axis=alt.Axis(format='%b %Y')),
        y = alt.Y('Value:Q', title = 'Normalized Value'),
        color = 'Symbol:N')

In [21]:
stock_vis = alt.Chart(long[long['Symbol'].isin(scaled_ser)]).mark_line().encode(
        x = 'Date:T',
        y = alt.Y('Value:Q', title = 'Normalized Value'),
        color = 'Symbol:N',
        opacity = alt.condition(select_ser, alt.value(1.0), alt.value(0.1))).add_params(select_ser)

In [22]:
(trend_vis + stock_vis).properties(title = 'AI Adjusted Stock Close Price & Google Search Interest')

alt.LayerChart(...)

In [23]:
series_cols = ["AAPL_ret", "Market_ret", "AI"]

chart1 = (
    alt.Chart(ff)
    .transform_fold(
        series_cols,
        as_=["series", "value"]
    )
    .mark_line()
    .encode(
        x=alt.X("date:T", title="Date"),
        y=alt.Y("value:Q", title="Value"),
        color=alt.Color("series:N", title="Series"),
        tooltip=[
            alt.Tooltip("date:T", title="Date"),
            alt.Tooltip("series:N", title="Series"),
            alt.Tooltip("value:Q", title="Value", format=".3f")
        ]
    )
    .properties(
        width=700,
        height=300,
        title="AAPL Return vs Market vs AI Trend"
    )
)

chart1

NameError: name 'ff' is not defined

In [24]:
# ---- Load stocks_monthly ----
stocks = pd.read_csv("stocks_monthly.csv")
stocks["Date"] = pd.to_datetime(stocks["Date"])

# Choose which columns to compare (adjust to your real column names)
price_cols = ["AAPL_Adj Close", "MSFT_Adj Close", "GOOG_Adj Close"]

chart2 = (
    alt.Chart(stocks)
    .transform_fold(
        price_cols,
        as_=["ticker", "price"]
    )
    .mark_line()
    .encode(
        x=alt.X("Date:T", title="Date"),
        y=alt.Y("price:Q", title="Adjusted Close Price"),
        color=alt.Color("ticker:N", title="Series"),
        tooltip=[
            alt.Tooltip("Date:T", title="Date"),
            alt.Tooltip("ticker:N", title="Series"),
            alt.Tooltip("price:Q", title="Price", format=".2f")
        ]
    )
    .properties(
        width=700,
        height=300,
        title="Stock Price Comparison"
    )
)

chart2


alt.Chart(...)

In [31]:
import pandas as pd
import altair as alt

alt.data_transformers.disable_max_rows()
alt.renderers.enable("default")

# Load trends
trends = pd.read_csv("trends_monthly.csv")
trends["date"] = pd.to_datetime(trends["date"])

# Pick the trend columns you actually have
trend_cols = ["AI", "ChatGPT", "Machine Learning", "OpenAI"]  # adjust if needed

# Legend selection to highlight specific terms
legend_sel = alt.selection_point(fields=["term"], bind="legend")

better_trend_chart = (
    alt.Chart(trends)
    .transform_fold(
        trend_cols,
        as_=["term", "value"]
    )
    .mark_line()
    .encode(
        x=alt.X("date:T", title="Date"),
        y=alt.Y("value:Q", title="Search Interest (0–100)"),
        color=alt.Color("term:N", title="Term"),
        opacity=alt.condition(legend_sel, alt.value(1.0), alt.value(0.15)),
        tooltip=[
            alt.Tooltip("date:T", title="Date"),
            alt.Tooltip("term:N", title="Term"),
            alt.Tooltip("value:Q", title="Index")
        ],
    )
    .add_selection(legend_sel)
    .properties(
        width=700,
        height=350,
        title="AI-related Google Trends (click legend to focus terms)"
    )
    .interactive()  # enables pan/zoom on axes
)

better_trend_chart



/var/folders/zz/ynlcz7_s0qs12pzd2n05s7tw0000gn/T/ipykernel_3808/1950698701.py:35: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use add_params instead.
  .add_selection(legend_sel)


alt.Chart(...)

In [29]:
import pandas as pd
import altair as alt

alt.data_transformers.disable_max_rows()
alt.renderers.enable("default")

# Load stock data
stocks = pd.read_csv("stocks_monthly.csv")
stocks["Date"] = pd.to_datetime(stocks["Date"])

# Columns to choose from (adjust if your names differ)
price_cols = [
    "AMZN_Adj Close",
    "META_Adj Close",
    "GOOGL_Adj Close",
    "NVDA_Adj Close",
    "MSFT_Adj Close"
]

# Param bound to a dropdown
stock_param = alt.param(
    name="Stock",  # this name is used in the filter expression below
    value=price_cols[0],
    bind=alt.binding_select(options=price_cols, name="Stock: ")
)

chart2 = (
    alt.Chart(stocks)
    .transform_fold(price_cols, as_=["ticker", "price"])
    # filter rows whose "ticker" matches the selected value
    .transform_filter("datum.ticker == Stock")
    .mark_bar(opacity=0.7, binSpacing=0)
    .encode(
        x=alt.X("price:Q", bin=alt.Bin(maxbins=30), title="Price"),
        y=alt.Y("count()", title="Frequency"),
        color=alt.Color("ticker:N", legend=None),
        tooltip=["ticker:N", "count()"]
    )
    .add_params(stock_param)
    .properties(
        width=700,
        height=300,
        title="Interactive Stock Price Distribution"
    )
)

chart2



alt.Chart(...)